<a href="https://colab.research.google.com/github/saivenkatreddy29/Building-LLM-from-Scratch/blob/main/Chapter_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from importlib.metadata import version
print("torch version", version("torch"))

torch version 2.5.1+cu121


In [2]:
import torch

inputs = torch.tensor(
  [[0.43, 0.15, 0.89], # Your     (x^1)
   [0.55, 0.87, 0.66], # journey  (x^2)
   [0.57, 0.85, 0.64], # starts   (x^3)
   [0.22, 0.58, 0.33], # with     (x^4)
   [0.77, 0.25, 0.10], # one      (x^5)
   [0.05, 0.80, 0.55]] # step     (x^6)
)

Mathemetical dot product

In [3]:
result = torch.zeros(len(inputs))
n = len(inputs)
query = inputs[1]
for i in range(n):
  result[i] = (sum(query*inputs[i]))
print(result)


tensor([0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865])


Using torch

In [4]:
import numpy as np
query = inputs[1]
attention_scores = torch.zeros(len(inputs))
n = len(inputs)
for i in range(n):
  attention_scores[i] = torch.dot(query,inputs[i])
attention_scores

tensor([0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865])

In [5]:
attention_scores.shape,inputs.shape

(torch.Size([6]), torch.Size([6, 3]))

Using mathematical method for normalizing

In [6]:
math_norm = attention_scores/sum(attention_scores)
math_norm

tensor([0.1455, 0.2278, 0.2249, 0.1285, 0.1077, 0.1656])

Using softmax written in python

In [7]:
import math
def naive_softmax(x):
  denom = 0
  for i in x:
    denom += math.exp(i)
  print(denom)
  # denom = sum(math.exp(x))
  return torch.exp(x)/denom
python_norm = naive_softmax(attention_scores)
print(python_norm)


18.745270996882674
tensor([0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581])


Using torch softmax

In [8]:
normalized_scores = torch.softmax(attention_scores, dim = -1)
normalized_scores

tensor([0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581])

Context vector for 2nd input we used as example

In [9]:
query = inputs[1]
context_vec_2 = torch.zeros(query.shape)
print(context_vec_2)
for i in range(n):
  # print(f'i:{inputs[i]}')
  context_vec_2 += normalized_scores[i] * inputs[i]
print(context_vec_2)

tensor([0., 0., 0.])
tensor([0.4419, 0.6515, 0.5683])


In [10]:
attn_score = inputs@inputs.T
print(attn_score)

tensor([[0.9995, 0.9544, 0.9422, 0.4753, 0.4576, 0.6310],
        [0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865],
        [0.9422, 1.4754, 1.4570, 0.8296, 0.7154, 1.0605],
        [0.4753, 0.8434, 0.8296, 0.4937, 0.3474, 0.6565],
        [0.4576, 0.7070, 0.7154, 0.3474, 0.6654, 0.2935],
        [0.6310, 1.0865, 1.0605, 0.6565, 0.2935, 0.9450]])


In [11]:
attn_weights = torch.softmax(attn_score, dim = -1)
print(attn_weights)

tensor([[0.2098, 0.2006, 0.1981, 0.1242, 0.1220, 0.1452],
        [0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581],
        [0.1390, 0.2369, 0.2326, 0.1242, 0.1108, 0.1565],
        [0.1435, 0.2074, 0.2046, 0.1462, 0.1263, 0.1720],
        [0.1526, 0.1958, 0.1975, 0.1367, 0.1879, 0.1295],
        [0.1385, 0.2184, 0.2128, 0.1420, 0.0988, 0.1896]])


In [12]:
print(attn_weights.sum(dim = -1))

tensor([1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000])


In [13]:
context_vector = attn_weights @ inputs
print(context_vector)

tensor([[0.4421, 0.5931, 0.5790],
        [0.4419, 0.6515, 0.5683],
        [0.4431, 0.6496, 0.5671],
        [0.4304, 0.6298, 0.5510],
        [0.4671, 0.5910, 0.5266],
        [0.4177, 0.6503, 0.5645]])
